# 理解嵌入层和线性层的区别

- PyTorch 中的嵌入层与执行矩阵乘法的线性层功能相同；我们使用嵌入层的原因是为了提高计算效率。
- 将通过 PyTorch 中的代码示例逐步了解这种关系。

In [3]:
import torch
print("PyTorch version: ", torch.__version__)

PyTorch version:  2.9.0+cpu


<br>
&nbsp;

## 使用 nn.Embedding(可训练的“词向量表”)

nn.Embedding是`PyTorch中的一个常用模块，其主要作用是将输入的整数序列转换为密集向量表示`。

In [16]:
# 假设我们有三个可训练的案例，它们可能代表 LLM 上下文中的标记 ID
idx = torch.tensor([2,3,1])
# 我们可以通过嵌入矩阵的行数量来决定最大的标记id为 ID+1
# 如果最高令牌 ID 为 3，则我们需要 4 行，分别对应可能的令牌 ID：0、1、2、3。
num_idx = max(idx)+1
print(idx)
print(num_idx
# 期望的嵌入维度是一个超参数
out_dim = 5

tensor([2, 3, 1])
tensor(4)


- 让我们实现一个简单的嵌入层：

In [17]:
# 我们使用随机种子是为了保证结果的可复现性，因为嵌入层中的权重是用小的随机值初始化的
torch.manual_seed(123)
# 输出4行5列
embedding = torch.nn.Embedding(num_idx,out_dim)

- 我们还可以选择性地查看嵌入权重：

In [8]:
embedding.weight

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.3035, -0.5880,  1.5810],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015],
        [ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953]], requires_grad=True)

- 然后我们可以使用嵌入层来获得 ID 为 1 的训练样本的向量表示：

In [9]:
embedding(torch.tensor([1]))

tensor([[ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

- 下面这张图展示了底层运行机制：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/1.png" width="600px">

- 类似地，我们可以使用嵌入层来获得 ID 为 2 的训练样本的向量表示：

In [10]:
embedding(torch.tensor([2]))

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315]],
       grad_fn=<EmbeddingBackward0>)

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/2.png" width="600px">

- 现在，让我们转换之前定义的所有训练样本：

In [11]:
idx = torch.tensor([2, 3, 1])
embedding(idx)

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

- 其底层原理仍然是相同的查找概念：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/3.png" width="700px">

<br>
&nbsp;

## 使用nn.Linear

nn.Linear() 是`PyTorch 中定义全连接层（也称为线性层、全连接层或密集层）的函数`。

- 现在，我们将证明上面的嵌入层与 PyTorch 中基于独热编码表示的 `nn.Linear` 层实现的功能完全相同。
- 首先，让我们将 token ID 转换为独热编码表示：

In [18]:
onehot = torch.nn.functional.one_hot(idx)
onehot

tensor([[0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0]])

- 下一步,初始化`Liner`层,该层执行矩阵乘法 $X W^\top$:

In [20]:
torch.manual_seed(123)
liner = torch.nn.Linear(num_idx,out_dim,bias= False)
liner.weight

Parameter containing:
tensor([[-0.2039,  0.0166, -0.2483,  0.1886],
        [-0.4260,  0.3665, -0.3634, -0.3975],
        [-0.3159,  0.2264, -0.1847,  0.1871],
        [-0.4244, -0.3034, -0.1836, -0.0983],
        [-0.3814,  0.3274, -0.1179,  0.1605]], requires_grad=True)

- 请注意，PyTorch 中的线性层也使用较小的随机权重进行初始化；为了将其与上面的 `Embedding` 层直接比较，我们必须使用相同的较小随机权重，这就是为什么我们在这里重新赋值它们的原因：

In [21]:
liner.weight = torch.nn.Parameter(embedding.weight.T)

- 现在我们可以对输入的独热编码表示使用线性层：

In [22]:
liner(onehot.float())

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]], grad_fn=<MmBackward0>)

- 正如我们所看到的，这与我们使用嵌入层时得到的结果完全相同：

In [23]:
embedding(idx)

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

- 其底层执行的是以下针对第一个训练样本的标记 ID 的计算：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/4.png" width="700px">

- 第二个训练样本的令牌 ID 为：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/5.png" width="700px">

- 由于每个独热编码行中除一个索引外的所有索引均为 0（这是设计使然），因此这种矩阵乘法本质上等同于查找独热元素。
- 这种对独热编码进行矩阵乘法的方法等价于嵌入层查找，但如果处理大型嵌入矩阵，效率可能会很低，因为存在大量浪费的零乘法。